# Analítica de datos
# Examen parcial 1
# 2018-02
# NOMBRE APELLIDO, código

El dataset que deben analizar contiene los registros del histórico de las personas a las que se les ha otorgado un crédito financiero, indicando quienes pudieron pagarlo sin problemas y quienes no.

La idea es poder predecir, utilizando un modelo de aprendizaje supervisado (KNN) y las variables independientes disponibles qué aspirantes a un crédito tienen mas o menos riesgo de pagar.

Las variables de la hoja de datos son las siguientes:
- ID: El ID del cliente en el banco. Numérico, debe ser positivo y único.
- PudoPagar: Indica si el cliente ha podido pagar el crédito sin problemas (1) o no (0). Variable categórica binaria.
- Estudiante: Indica si el cliente es estudiante (Si) o no lo es (No). Variable categórica binaria.
- Ingresos: Indica la cantidad del crédito que aún adeuda el cliente. Deben ser valores numéricos positivos o nulos.
- Cuenta: Indica el dinero que tiene en la cuenta del banco. Deben ser valores numéricos.

## Entendimiento de los datos, limpieza

El archivo "creditos_examen.csv" contiene el dataset que tienen que analizar.
Se recomienda abrirlo primero en un lector de archivos planos para entender preliminarmente su formato y así poderlo cargar adecuadamente con Python.

Teniendo en cuenta el tipo de problema en cuestión (clasificación o regresión), realice un análisis exploratorio de los datos estableciendo el baseline, verificando la calidad de los datos (tipos de las variables, valores inválidos, excepciones, valores faltantes, etc.), utilizando gráficos para poder entender las distribuciones de los datos e identificar posibles problemas.

#### Puntos a desarrollar

- Carguen el archivo en memoria y exploren los datos, tratando de identificar problemas e inconsistencias teniendo en cuenta el diccionario de datos y el contexto del problema. (0.7)

- Obtengan el modelo de referencia (Baseline) y calculen sus métricas de evaluación. (0.3)

- Limpien los datos arreglando los problemas, eliminando columnas, eliminando registros, o ignorándolos, justificando sus decisiones. 
Una vez resueltos los problemas encontrados, indique la siguiente información: ID del registro, Tipo de ProblemaResolución. (1.2)

## Modelamiento con K-NN

#### Puntos a desarrollar:

- Escogiendo un valor de los parámetros al azar, construyan un modelo K-NN inicial sobre los datos limpios y calcule las métricas de accuracy, error, kappa, precision, recall, especifidad, y F-measure. Analice la calidad del modelo interpretando las métricas (1.0).

- Teniendo en cuenta las métricas más adecuadas para el problema, y siguiendo un protocolo de evaluación adecuado, establezca el mejor valor de los parámetros del modelo. (1.5)

- ¿El valor de los parámetros inicialmente escogido estaba en una situación de overfitting o de underfitting? Explique su respuesta. (0.5)

# Desarrollo de los puntos

In [1]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import KFold, cross_val_score #protocolo de evaluación
from sklearn import neighbors, datasets, metrics
from sklearn import preprocessing 
import seaborn as sns

# EDA

Cargamos los datos en memoria

In [3]:
df = pd.read_csv("creditos_examen.csv", sep=";")
df.head(25)

,ID,PudoPagar,Estudiante,Ingresos,Cuenta
0,-1,0,No,3196574,358943
1,-1,0,No,4944040,3857756
2,1,0,No,4436163,3647632
3,2,0,Si,1210613,4085902
4,3,0,No,3176714,5367746
5,4,0,No,3570449,2646253
6,5,0,No,3846350,3928279
7,6,0,Si,749156,4597943
8,7,0,No,2490523,4127567
9,8,0,Si,1760045,4043338


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
ID            10000 non-null int64
PudoPagar     10000 non-null int64
Estudiante    9996 non-null object
Ingresos      10000 non-null int64
Cuenta        10000 non-null int64
dtypes: int64(4), object(1)
memory usage: 390.7+ KB


In [7]:
df.describe(include="all")

,ID,PudoPagar,Estudiante,Ingresos,Cuenta
count,10000.000000,10000.000000,9996,1.000000e+04,1.000000e+04
unique,NaN,NaN,2,NaN,NaN
top,NaN,NaN,No,NaN,NaN
freq,NaN,NaN,7052,NaN,NaN
mean,5000.423000,0.033300,NaN,3.036625e+06,1.433952e+07
std,2887.023398,0.179428,NaN,7.028938e+07,1.016575e+09
min,-1.000000,0.000000,NaN,-4.288864e+09,-2.247063e+07
25%,2500.750000,0.000000,NaN,1.823537e+06,2.408656e+06
50%,5000.500000,0.000000,NaN,3.209805e+06,4.118185e+06
75%,7500.250000,0.000000,NaN,4.281578e+06,5.831542e+06


In [10]:
df.dtypes

ID             int64
PudoPagar      int64
Estudiante    object
Ingresos       int64
Cuenta         int64
dtype: object

# Limpieza

## Eliminar columna de ID's

Elimino la columna de IDs ya que es un dato irrelevante

In [11]:
df = df.drop(['ID'], axis=1)
df.head()

,PudoPagar,Estudiante,Ingresos,Cuenta
0,0,No,3196574,358943
1,0,No,4944040,3857756
2,0,No,4436163,3647632
3,0,Si,1210613,4085902
4,0,No,3176714,5367746


Cambiamos la variable objetivo a categorica porque vamos a hacer clasificación y no a predecir

## Variable objetivo

In [13]:
df.PudoPagar = df.PudoPagar.astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
PudoPagar     10000 non-null object
Estudiante    9996 non-null object
Ingresos      10000 non-null int64
Cuenta        10000 non-null int64
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


## Valores faltantes

In [14]:
# Obtenemos los na y los utilizamos indices para obtenerlos del dataframe
df[df.Estudiante.isna()]

,PudoPagar,Estudiante,Ingresos,Cuenta
759,0,NaN,2538070,1924989
779,0,NaN,4119537,5096245
799,0,NaN,3377683,50944
819,0,NaN,2721295,474662


In [15]:
df = df.dropna()

In [16]:
df.describe(include="all")

,PudoPagar,Estudiante,Ingresos,Cuenta
count,9996,9996,9.996000e+03,9.996000e+03
unique,2,2,NaN,NaN
top,0,No,NaN,NaN
freq,9663,7052,NaN,NaN
mean,NaN,NaN,3.036563e+06,1.434450e+07
std,NaN,NaN,7.030345e+07,1.016778e+09
min,NaN,NaN,-4.288864e+09,-2.247063e+07
25%,NaN,NaN,1.823380e+06,2.409284e+06
50%,NaN,NaN,3.209805e+06,4.118502e+06
75%,NaN,NaN,4.281987e+06,5.833031e+06
